# 📓 Real-World Agentic AI: The Customer Support Bot

Welcome! In this demonstration, we'll build a simple but realistic AI system. Our goal is to create a **Customer Support Bot** that can check a user's order status.

We'll move beyond simple dictionaries and functions to show how these systems are structured in the real world.

**Our Plan:**
1.  **The Agent Registry**: We'll use a `JSON` file, which is how configurations are often stored.
2.  **The Tool (Order API)**: A class that represents an external company database.
3.  **The MCP Simulation**: The agent and tool will communicate using `JSON text strings`, mimicking how real web APIs talk to each other.



---
## 1. The Agent Registry: A `JSON` File

In a real project, you wouldn't store your agent definitions inside your Python code. You'd use an external configuration file. `JSON` is a perfect choice because it's easy for both humans and machines to read.

Let's create our `agent_registry.json` file.

In [ ]:
import json

# Define the agent's properties in a Python dictionary first
agent_definition = {
    "support_agent_v1": {
        "agent_id": "support_agent_v1",
        "description": "A bot that can fetch the status of a customer's order.",
        "version": "1.1",
        "capabilities": {
            "task": "get_order_status",
            "required_tool": "OrderAPITool" # The name of the tool it needs
        }
    }
}

# Now, write this dictionary to a JSON file
file_path = 'agent_registry.json'
with open(file_path, 'w') as f:
    json.dump(agent_definition, f, indent=4)

print(f"✅ Agent Registry created at '{file_path}'")

After running the cell above, you will have a new file named `agent_registry.json` in the same directory as your notebook. This file is our professional-looking registry!

---
## 2. The Tool and the Protocol (MCP)

Now for the most important part. We will simulate our **Model Context Protocol (MCP)**. Our rule is simple: **all communication must be in the form of a JSON string.**

### The Tool: An External Order Database
First, let's define the tool our agent will use. This class simulates your company's order database. It will only accept requests and give responses as JSON text.

In [ ]:
class OrderAPITool:
    """
    This simulates a real, external tool, like a database API.
    It strictly communicates by receiving and returning JSON strings.
    """
    def __init__(self):
        # A fake database of orders for our demonstration
        self._database = {
            "ORD123": {"status": "Shipped", "item": "Laptop"},
            "ORD456": {"status": "Processing", "item": "Keyboard"},
            "ORD789": {"status": "Delivered", "item": "Mouse"},
        }
    
    def call_tool(self, json_request: str) -> str:
        """
        This is the tool's only entry point. It follows our MCP rule.
        """
        print(f"⚙️ TOOL: Received raw JSON request: {json_request}")
        
        # --- Tool-side Deserialization ---
        # The tool converts the JSON string back into a Python dictionary
        try:
            request_data = json.loads(json_request)
            order_id = request_data.get("order_id")
        except (json.JSONDecodeError, AttributeError):
            response = {"success": False, "error": "Invalid JSON format"}
            return json.dumps(response)

        # --- Tool Logic ---
        # The tool performs its actual task
        if order_id in self._database:
            order_info = self._database[order_id]
            response = {"success": True, "data": order_info}
        else:
            response = {"success": False, "error": "Order ID not found"}
        
        # --- Tool-side Serialization ---
        # The tool converts its Python dictionary response back into a JSON string
        json_response = json.dumps(response)
        print(f"⚙️ TOOL: Sending back raw JSON response: {json_response}")
        return json_response

print("✅ OrderAPITool defined.")

---
## 3. The Agent in Action

Now we'll build the main application. This code will act as our AI agent. It will use the registry to find out what to do and then use the MCP-compliant tool to do it.

In [ ]:
class SupportAgent:
    """Our main AI Agent."""
    
    def __init__(self, registry_path: str):
        # 1. The agent loads its configuration from the registry file on startup.
        with open(registry_path, 'r') as f:
            self.registry = json.load(f)
        print("🤖 AGENT: Registry loaded.")
        
        # The agent has access to a list of available tools.
        self.tool_library = {
            "OrderAPITool": OrderAPITool()
        }
        print("🤖 AGENT: Tool library initialized.")

    def perform_task(self, agent_id: str, order_id: str):
        """The main logic loop for the agent."""
        
        print(f"\n🚀 AGENT: Initiating task 'get_order_status' for Order ID: {order_id}")
        
        # 2. Discovery: Look up its own instructions in the registry.
        agent_info = self.registry.get(agent_id)
        if not agent_info:
            print(f"❌ AGENT: I don't know who I am! Agent '{agent_id}' not in registry.")
            return

        tool_name = agent_info["capabilities"]["required_tool"]
        target_tool = self.tool_library.get(tool_name)
        print(f"🤖 AGENT: I need to use the '{tool_name}'.")

        # 3. Prepare MCP Request: The agent prepares its request as a Python dictionary.
        request_payload = {
            "task": "get_order_status",
            "order_id": order_id
        }
        
        # --- Agent-side Serialization ---
        # It converts the dictionary to a JSON string before "sending" it.
        json_request = json.dumps(request_payload)
        
        # 4. Execute using MCP: The agent calls the tool with the JSON string.
        json_response = target_tool.call_tool(json_request)
        
        # --- Agent-side Deserialization ---
        # It gets a JSON string back and converts it into a dictionary it can use.
        response_data = json.loads(json_response)
        
        # 5. Process Final Result
        if response_data.get("success"):
            status = response_data["data"]["status"]
            item = response_data["data"]["item"]
            print(f"\n✅ SUCCESS! The status for order '{order_id}' ({item}) is: '{status}'.")
        else:
            error = response_data.get("error")
            print(f"\n❌ FAILED. The tool reported an error: '{error}'.")


# --- Let's run our Support Bot! ---
support_bot = SupportAgent(registry_path='agent_registry.json')
support_bot.perform_task(agent_id="support_agent_v1", order_id="ORD456")

### Let's try another one that will fail:

In [ ]:
# --- Running again with an Order ID that doesn't exist ---
support_bot.perform_task(agent_id="support_agent_v1", order_id="ORD999")

---

## 4. Conclusion: Why Was This Powerful?

This example, while still simple, is a much more accurate model of a real agentic system.

1.  **Clear Separation**: The `agent_registry.json` file is totally separate from the code. You can update the agent's description or version without touching the Python script.
2.  **Formal Protocol (MCP)**: By forcing communication through `JSON strings`, we simulated a real API. The agent doesn't need to know *how* the tool works, and the tool doesn't need to know *who* is calling it. They just need to agree on the message format.
3.  **Scalability**: To add a new "Process Refund" agent, you would simply:
    * Add a new entry to the `agent_registry.json` file.
    * Build a new `RefundAPITool` class that also "speaks" JSON.
    * The core logic of the `SupportAgent` wouldn't need to change at all.

This **decoupled, protocol-driven architecture** is the key to building robust AI systems that can grow in complexity without breaking.